In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state:JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content
    return {'joke':response}

In [5]:
def generate_explanation(state:JokeState):
    prompt = f'Write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation':response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explanation')
graph.add_edge('generate_explanation',END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer = checkpointer)

In [15]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({'topic':'ball'},config=config1)

{'topic': 'ball',
 'joke': "Why did the basketball go to therapy? \n\nBecause it had too many issues it couldn't bounce back from!",
 'explanation': 'This joke is a play on words, as "bounce back" is a term commonly associated with basketball. The joke is suggesting that the basketball went to therapy because it had too many emotional or personal issues that it couldn\'t "bounce back" from on its own. This adds a humorous and unexpected twist to the idea of an inanimate object seeking therapy.'}

In [16]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'ball', 'joke': "Why did the basketball go to therapy? \n\nBecause it had too many issues it couldn't bounce back from!", 'explanation': 'This joke is a play on words, as "bounce back" is a term commonly associated with basketball. The joke is suggesting that the basketball went to therapy because it had too many emotional or personal issues that it couldn\'t "bounce back" from on its own. This adds a humorous and unexpected twist to the idea of an inanimate object seeking therapy.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d822d-e005-6c7c-800a-612ed4244e93'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2025-12-13T12:54:31.353845+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d822d-d7f8-6be3-8009-788ebe6da160'}}, tasks=(), interrupts=())

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'ball', 'joke': "Why did the basketball go to therapy? \n\nBecause it had too many issues it couldn't bounce back from!", 'explanation': 'This joke is a play on words, as "bounce back" is a term commonly associated with basketball. The joke is suggesting that the basketball went to therapy because it had too many emotional or personal issues that it couldn\'t "bounce back" from on its own. This adds a humorous and unexpected twist to the idea of an inanimate object seeking therapy.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d822d-e005-6c7c-800a-612ed4244e93'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2025-12-13T12:54:31.353845+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d822d-d7f8-6be3-8009-788ebe6da160'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'ball', 'joke': "Why did the basketball go to therapy? 

In [18]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'beer'}, config=config2)

{'topic': 'beer',
 'joke': 'Why did the beer go to the party alone? \n\nBecause it just wanted to have a "brew-tiful" time!',
 'explanation': 'This joke is a play on words using the term "brew-tiful", which is a combination of "brew" (referring to beer) and "beautiful". The joke implies that the beer went to the party alone because it simply wanted to enjoy itself and have a good time, or a "brew-tiful" time, without needing anyone else\'s company. It\'s a light-hearted and punny way to explain why the beer chose to attend the party solo.'}

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'girls', 'joke': 'Why did the girl bring a ladder to the bar? \nBecause she heard the drinks were on the house!', 'explanation': 'This joke plays on the phrase "drinks on the house," which means that someone is offering to pay for the drinks. In this case, the girl interpreted it literally and thought that the drinks were located on the roof of the bar, so she brought a ladder to reach them. It\'s a silly and clever play on words that brings humor to the situation.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d8226-8867-6b38-8002-a9eb5ba79cf3'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-13T12:51:14.261674+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d8226-7fc3-6b5a-8001-1fd26536c03a'}}, tasks=(), interrupts=())

In [13]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'beer', 'joke': "Why did the beer go to the party alone?\n\nBecause it didn't want any bad influence from his brew crew!", 'explanation': 'This joke plays on the idea that beer is typically consumed in a social setting, often with a group of friends or "brew crew." By saying that the beer went to the party alone, the implication is that the beer wanted to avoid any negative influence from its friends and ensure that it behaved responsibly at the party. It\'s a light-hearted way to poke fun at the idea of peer pressure and the importance of making good decisions even when surrounded by others who may not always have your best interests at heart.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0d822b-138d-6d45-8002-d00e3f42d080'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-13T12:53:16.226679+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpo

Time Travel

In [19]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0d8226-745f-6548-8000-0e65147083ac"}})

StateSnapshot(values={'topic': 'girls'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0d8226-745f-6548-8000-0e65147083ac'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-13T12:51:12.161089+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d8226-7453-6d1b-bfff-9aeb0568a41c'}}, tasks=(PregelTask(id='f420bc9f-61dc-9fe7-e10e-c5b684d26cfd', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the girl bring a ladder to the bar? \nBecause she heard the drinks were on the house!'}),), interrupts=())

In [20]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0d8226-745f-6548-8000-0e65147083ac"}})

{'topic': 'girls',
 'joke': 'Why did the girl bring a ladder to the bar? \n\nBecause she heard the drinks were on the house!',
 'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar setting, "on the house" typically means that the drinks are free or being paid for by the establishment. However, in this case, the girl takes the phrase literally and brings a ladder to reach the drinks that are supposedly on top of the house. The joke is humorous because it involves a clever and unexpected interpretation of a common phrase.'}

Updating State

In [21]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0d8226-745f-6548-8000-0e65147083ac", "checkpoint_ns": ""}}, {'topic':'jalebi'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0d8237-cfc4-6644-8001-d013c6039cdf'}}

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'jalebi'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d8237-cfc4-6644-8001-d013c6039cdf'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-13T12:58:58.084791+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d8226-745f-6548-8000-0e65147083ac'}}, tasks=(PregelTask(id='8ca81613-b850-ba30-094d-bbe8bfbb8dc3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'girls', 'joke': 'Why did the girl bring a ladder to the bar? \n\nBecause she heard the drinks were on the house!', 'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar setting, "on the house" typically means that the drinks are free or being paid for by the establishment. However, in this case, the girl ta

In [25]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0d8237-cfc4-6644-8001-d013c6039cdf"}})

{'topic': 'jalebi',
 'joke': 'Why did the jalebi go to school?\nBecause it wanted to learn how to twist and turn like a pro!',
 'explanation': 'This joke plays on the idea of jalebi, a popular Indian sweet that is known for its twisted and circular shape. By saying that the jalebi went to school to learn how to twist and turn like a pro, it is suggesting that the jalebi wanted to improve its twisting and turning skills, as if it were a student learning a new skill in school. This humorous twist adds a playful and lighthearted element to the joke.'}

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'jalebi', 'joke': 'Why did the jalebi go to school?\nBecause it wanted to learn how to twist and turn like a pro!', 'explanation': 'This joke plays on the idea of jalebi, a popular Indian sweet that is known for its twisted and circular shape. By saying that the jalebi went to school to learn how to twist and turn like a pro, it is suggesting that the jalebi wanted to improve its twisting and turning skills, as if it were a student learning a new skill in school. This humorous twist adds a playful and lighthearted element to the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d823c-7a7c-66d6-8003-ab8cea2a1a51'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-12-13T13:01:03.360160+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d823c-7027-6e13-8002-a7f4f0abc911'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'j

Fault Tolerance

In [27]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [28]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [29]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [30]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))